# Demo of CW-Tiler
This demo will walk through the use of the tiler to create utm tiles with the SpaceNet Data Repository.

We will be taking advantage of cloud optimized geotiffs.  For more information about SpaceNet visit https://spacenetchallenge.github.io/



In [ ]:
# Import base tools

## Note, for mac osx compatability import something from shapely.geometry before importing fiona or geopandas
## https://github.com/Toblerity/Shapely/issues/553  * Import shapely before rasterio or fioana
from shapely import geometry
import rasterio
import random
from cw_tiler import main
from cw_tiler import utils
from cw_tiler import vector_utils
import numpy as np
import os
from tqdm import tqdm
# Setting Certificate Location for Ubuntu/Mac OS locations (Rasterio looks for certs in centos locations)
os.environ['CURL_CA_BUNDLE']='/etc/ssl/certs/ca-certificates.crt'

In [ ]:
## give location to SpaceNet 8-Band PanSharpened Geotiff on s3

#spacenetPath = "s3://spacenet-dataset/AOI_2_Vegas/srcData/rasterData/AOI_2_Vegas_MUL-PanSharpen_Cloud.tif"
#spacenetPath = "/nfs/data/Datasets/CosmiQ_General_Study/AOI_6_Atlanta/srcData/rasterData/MUL-PanSharpen/057341085010_01_assembley_MULPan.tif"
spacenetPath = "/home/dlindenbaum/057341085010_01_assembley_MULPan.tif"
osm_labels_path = "/nfs/data/Datasets/CosmiQ_General_Study/Atlanta_GA/Atlanta_20121020-26_UFO/VEC/BLD2/atlanta_2d_buildings.shp"

In [ ]:
%%time
## Prep files for UTM
with rasterio.open(spacenetPath) as src:

    # Get Lat, Lon bounds of the Raster (src)
    wgs_bounds = utils.get_wgs84_bounds(src)
    
    # Use Lat, Lon location of Image to get UTM Zone/ UTM projection
    utm_crs = utils.calculate_UTM_crs(wgs_bounds)
    
    # Calculate Raster bounds in UTM coordinates 
    utm_bounds = utils.get_utm_bounds(src, utm_crs)

In [ ]:
%%time
## read vector file
gdf = vector_utils.read_vector_file(osm_labels_path)
gdf.head()
gdf_utm = vector_utils.transformToUTM(gdf, utm_crs=utm_crs)

In [ ]:
%%time
#print(gdf_utm.total_bounds)
#utm_bounds

geoBounds = geometry.box(*gdf_utm.total_bounds)
rasterBounds = geometry.box(*utm_bounds)

interBounds = geoBounds.intersection(rasterBounds)
interBounds.bounds


In [ ]:
# open s3 Location

# Each grid starting point will be spaced 400m apart
stride_size_meters = 400

# Each grid cell will be 400m on a side
cell_size_meters   = 400

# Specify the number of pixels in a tile cell_size_meters/tile_size_pixels == Pixel_Size_Meters
tile_size_pixels   = 800

dataLocation = "AOI_6_Atlanta"
imagePrefix = "AOI_6_Atlanta_MUL_{}_{}"
with rasterio.open(spacenetPath) as src:

    
    src_profile = src.profile
    # Generate list of cells to read from utm_bounds 
    cells_list = main.calculate_analysis_grid(interBounds.bounds, stride_size_meters=stride_size_meters, cell_size_meters=cell_size_meters)

    # select random cell
    for cell_selection in tqdm(cells_list):
        #random_cell = random.choice(cells_list)
        ll_x, ll_y, ur_x, ur_y = cell_selection


        # Get Tile from bounding box
        tile, mask, window_transform = main.tile_utm(src, ll_x, ll_y, ur_x, ur_y, indexes=None, tilesize=tile_size_pixels, nodata=None, alpha=None,
                     dst_crs=utm_crs)

        #print(np.shape(tile))

        ## Get Vector Information from bounding box
        small_gdf = vector_utils.vector_tile_utm(gdf_utm, tile_bounds=[ll_x, ll_y, ur_x, ur_y])
        if not small_gdf.empty:
            small_gdf.to_file(os.path.join(dataLocation, imagePrefix.format(int(ll_x), int(ll_y))+"_label.geojson"), driver='GeoJSON')
        
        else:
            open(os.path.join(dataLocation, imagePrefix.format(int(ll_x), int(ll_y))+"_label.geojson"), 'a').close()
        
        
        img = vector_utils.rasterize_gdf(small_gdf,
                                         src_shape=(tile_size_pixels, tile_size_pixels),
                                         src_transform=window_transform,
                                    )
        
        # update src_profile for writing new img
        dst_profile = src_profile
        dst_profile.update({'transform': window_transform,
                    'crs': utm_crs,
                    'width': tile_size_pixels,
                    'height': tile_size_pixels,
                            'count': 1,
                            'dtype': rasterio.uint8

                   })
        ## write label to tiff
        with rasterio.open(os.path.join(dataLocation, imagePrefix.format(int(ll_x), int(ll_y))+"_label.tif"),
                           'w',
                           **dst_profile) as dst:

            dst.write(img, indexes=1)
        
        ## write image to geotiff
        dst_profile = src_profile
        dst_profile.update({'transform': window_transform,
                    'crs': utm_crs,
                    'width': tile_size_pixels,
                    'height': tile_size_pixels,
                            'count': 8,
                            'dtype': rasterio.uint16
                   })
        
        with rasterio.open(os.path.join(dataLocation, imagePrefix.format(int(ll_x), int(ll_y))+"_image.tif"), 'w',
                           **dst_profile) as dst:

            dst.write(tile)

In [ ]:
%%time
# open s3 Location # concurrency test
import concurrent.futures


def processChip(data_gen):
    
    return data_gen


# Each grid starting point will be spaced 400m apart
stride_size_meters = 400

# Each grid cell will be 400m on a side
cell_size_meters   = 400

# Specify the number of pixels in a tile cell_size_meters/tile_size_pixels == Pixel_Size_Meters
tile_size_pixels   = 800
num_workers = 2
dataLocation = "AOI_6_Atlanta"
imagePrefix = "AOI_6_Atlanta_MUL_{}_{}"
with rasterio.open(spacenetPath) as src:

    
    src_profile = src.profile
    # Generate list of cells to read from utm_bounds 
    cells_list = main.calculate_analysis_grid(interBounds.bounds, stride_size_meters=stride_size_meters, cell_size_meters=cell_size_meters)
    cells_list = cells_list[0:100]
    # tile, mask, window_transform = data_gen
    data_gen = (main.tile_utm(src, *cell, indexes=None, tilesize=tile_size_pixels, nodata=None, alpha=None,
                     dst_crs=utm_crs) for cell in cells_list)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        
        for cell, result in zip(cells_list, executor.map(processChip, data_gen)):
    
            
        
            print(cell)
            ## write image to geotiff
            dst_profile = src_profile
            dst_profile.update({'transform': result[2],
                        'crs': utm_crs,
                        'width': tile_size_pixels,
                        'height': tile_size_pixels,
                                'count': 8,
                                'dtype': rasterio.uint16
                       })

            ll_x, ll_y, ur_x, ur_y = cell
            with rasterio.open(os.path.join(dataLocation, imagePrefix.format(int(ll_x), int(ll_y))+"_image.tif"), 'w',
                               **dst_profile) as dst:

                dst.write(result[0])
    
    

In [ ]:
with concurrent.futures.ThreadPoolExecutor(
                    max_workers=num_workers
                ) as executor:

                    # We map the compute() function over the raster
                    # data generator, zip the resulting iterator with
                    # the windows list, and as pairs come back we
                    # write data to the destination dataset.
                    for window, result in zip(
                        windows, executor.map(compute, data_gen)
                    ):
                        dst.write(result, window=window)

In [ ]:
%matplotlib inline
# Plot 1st Band of the Tile 
from  matplotlib import pyplot as plt
import numpy as np
#plt.imshow(img)
plt.imshow(tile[1,:,:]) 
if not small_gdf.empty:
    small_gdf.plot()

In [ ]:
    ## Set for specific tile in Las Vegas
    utm_crs = utils.calculate_UTM_crs([-115.30170, 36.15604, -115.30170, 36.15604])
    gdf = vector_utils.read_vector_file(geojsonPath)
    gdf.head()
    gdf = vector_utils.transformToUTM(gdf, utm_crs=utm_crs)

    utmX, utmY = 658029, 4006947
    ll_x = utmX
    ur_x = utmX + 500
    ll_y = utmY
    ur_y = utmY + 500
    stride_size_meters = 300
    cell_size_meters = 400
    tile_size_pixels = 1600

    address = spacenetPath

    with rasterio.open(address) as src:

        src_profile = src.profile


        tile, mask, window_transform = main.tile_utm(src, ll_x, ll_y, ur_x, ur_y,
                                                     indexes=None,
                                                     tilesize=tile_size_pixels,
                                                     nodata=None,
                                                     alpha=None,
                                                     dst_crs=utm_crs
                                                     )

        print(np.shape(tile))
        assert np.shape(tile) == (8, tile_size_pixels, tile_size_pixels)

        small_gdf = vector_utils.vector_tile_utm(gdf, tile_bounds=[ll_x, ll_y, ur_x, ur_y])
        print(small_gdf.shape)
        small_gdf.to_file(os.path.join(PREFIX, "testTiff_Label.geojson"), driver='GeoJSON')
        img = vector_utils.rasterize_gdf(small_gdf,
                                         src_shape=(tile_size_pixels, tile_size_pixels),
                                         src_transform=window_transform,
                                    )
        print("Label Count Burn {}:".format(np.sum(img)))
        assert img.shape == (tile_size_pixels, tile_size_pixels)


        dst_profile = src_profile
        dst_profile.update({'transform': window_transform,
                    'crs': utm_crs,
                    'width': tile_size_pixels,
                    'height': tile_size_pixels,
                            'count': 1,
                            'dtype': rasterio.uint8

                   })

        with rasterio.open(os.path.join(PREFIX, "testTiff_Label.tif"), 'w',
                                        **dst_profile) as dst:

            dst.write(img, indexes=1)

        dst_profile = src_profile
        dst_profile.update({'transform': window_transform,
                    'crs': utm_crs,
                    'width': tile_size_pixels,
                    'height': tile_size_pixels,
                            'count': 8,
                            'dtype': rasterio.uint16
                   })

        with rasterio.open(os.path.join(PREFIX, "testTiff_Image.tif"), 'w',
                           **dst_profile) as dst:

            dst.write(tile)

In [ ]:
import geopandas as gpd

In [ ]:
small_gdf = gpd.GeoDataFrame(geometry=[])

In [ ]:
small_gdf.area